![imagenes](logo.png)

# Boxplots

Aprendamos un tipo de gráfico que muy útil para la captura de datos atípicos: gráficos de cajas (boxplots).

Se trata de gráficos creados por Tukey en 1977, aplicado a variables aleatorias continuas. 

El procedimiento para construirlos es el siguiente: 

1. **Obtener los cuartiles:** ordenamos los datos de menor a mayor para ubicar los *cuartiles* $Q_1$, $Q_2$ y $Q_3$.

El primer cuartil, $Q_1$, es el valor que deja por debajo de sí al 25% de los datos.

El segundo cuartil, $Q_2$, es el valor que deja por debajo de sí al 50% de los datos. Es decir, la mediana.

El tercer cuartil, $Q_3$, es el valor que deja por debajo de sí al 75% de los datos.


En otras palabras, $Q_2$ es la mediana de los datos; $Q_1$ es la mediana de los datos que quedan debajo de $Q_2$; y $Q_3$ es la mediana de los datos que quedan por arriba de $Q_2$.

Una vez obtenidos los cuartiles, dibujamos la caja desde $Q_1$ hasta $Q_3$, además de una línea en $Q_2$:

<img src="im016.png" style="display:block; margin:auto;">

2. **Calcular el Rango intercuartílico:** Esto es simplemente $RI=Q_3-Q_1$

Una vez que tenemos el $RI$, calculamos las *barreras* como $Q_1-1.5RI$ y $Q_3+1.5RI$. Posteriormente graficamos dos lineas punteadas en las posiciones de las barreras.

<img src="im017.png" style="display:block; margin:auto;">

3. **Construcción de los bigotes:** Ubicamos el mayor valor que no sobrepasa a la barrera superior y el menor valor que no queda debajo de la barrera inferior. Dibujamos una linea desde la caja hasta esos dos valores. Los dos segmentos determinados se llaman *bigotes*.

<img src="im018.png" style="display:block; margin:auto;">

4. **Outliers.** Se añaden los valores superiores a la barrera superior e inferiores a la barrera inferior. Estos son considerados como datos atípicos.

<img src="im019.png" style="display:block; margin:auto;">

Usualmente, las barreras no se grafican. Además de que los bigotes no se dibujan con los segmentos "extremos". Es decir, normalmente el boxplot anterior se visualiza así:

<img src="im020.png" style="display:block; margin:auto;">

Veamos cómo implementar los boxplots: Consideremos las edades de alumnos_scidata.csv.

```r
################################################################
########################      EN R       #######################
################################################################
library(tidyverse)

alumnos = read.csv("alumnos_scidata.csv")
edades = alumnos$Edad
edades = edades[!is.na(edades)]

resumen = summary(edades)
Q1 = resumen[2]
mediana = resumen[3]
Q3 = resumen[5]

RI = Q3-Q1
Barrera_inf = Q1-1.5*RI
Barrera_sup = Q3+1.5*RI

atipicos_boxplot = edades[edades < Barrera_inf | edades > Barrera_sup]
alumnos[alumnos$Edad %in% atipicos_boxplot,]

ggplot(data=alumnos) + geom_boxplot(mapping=aes(y=Edad))
```


```python
################################################################
########################    EN PYTHON    #######################
################################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

alumnos = pd.read_csv("alumnos_scidata.csv")
edades = alumnos["Edad"]
edades = edades[~np.isnan(edades)]

resumen = alumnos["Edad"].describe()

Q1 = resumen[4]
Q2 = resumen[5]
Q3 = resumen[6]

RI = Q3 - Q1
Barrera_inf = Q1 - 1.5 * RI
Barrera_sup = Q3 + 1.5 * RI

atipicos_boxplot = edades[(edades < Barrera_inf) | (edades > Barrera_sup)]
alumnos[alumnos["Edad"].isin(atipicos_boxplot)]

# Graficar el boxplot
plt.figure(figsize=(6, 4))
plt.boxplot(edades, vert=True, patch_artist=True)
plt.ylabel("Edad")
plt.title("Diagrama de Caja de Edades")
plt.show()

##############################################################
##############################################################
```

## Boxplot Ajustado con Medcouple

El **boxplot ajustado para asimetría** utiliza el **medcouple (MC)** para modificar la detección de valores atípicos en distribuciones asimétricas. 

### **¿Qué es el medcouple (MC)?**
El **medcouple** es una medida robusta de **asimetría** basada en los valores centrales de la distribución.
- Si **MC > 0**, la distribución tiene **asimetría positiva** (cola larga a la derecha).
- Si **MC < 0**, la distribución tiene **asimetría negativa** (cola larga a la izquierda).
- Si **MC ≈ 0**, la distribución es aproximadamente **simétrica**.

A diferencia de la **asimetría de Pearson**, el medcouple es **más robusto**, ya que no depende de la media y la varianza, sino de la **mediana** y la **dispersión relativa de los datos**.

Se calcula de la siguiente manera: si los datos son $x_1,x_2,..,x_n$ y $Me$ es la mediana, para cada par de datos $x_i,x_j$ que cumplan $x_i\le Me\le x_j$ y $x_i\neq x_j$ se define $$h(x_i,x_j)=\frac{(x_j-Me)-(Me-x_i)}{x_j-x_i}$$ y la Medcouple se calcula como $MC=$mediana de todos los $h(x_i,x_j)$. Se trata de un estimador robusto de la simetría.


### **Modificación del Boxplot Tradicional**
En un **boxplot tradicional**, los valores atípicos se detectan con:

$$Q_1 - 1.5 \times IQR, \quad Q_3 + 1.5 \times IQR$$

En el **boxplot ajustado con medcouple**, se ajustan los coeficientes en función del **MC**:

- Si **MC > 0** (asimetría positiva), el límite superior se **expande** y el inferior se **contrae**.

$$L_{\text{inf}} = Q_1 - 1.5 \times e^{-3.5 \times MC} \times IQR$$

$$L_{\text{sup}} = Q_3 + 1.5 \times e^{4 \times MC} \times IQR$$
  
- Si **MC < 0** (asimetría negativa), el límite inferior se **expande** y el superior se **contrae**.

$$L_{\text{inf}} = Q_1 - 1.5 \times e^{-4 \times MC} \times IQR$$

$$L_{\text{sup}} = Q_3 + 1.5 \times e^{3.5 \times MC} \times IQR$$


El boxplot ajustado se utiliza cuando los datos son **altamente asimétricos**, ya que el boxplot tradicional puede generar demasiados valores atípicos en la cola larga. En **big data**, ya que el medcouple es **robusto y eficiente** para grandes muestras.

Veamos ahora las implementaciones:

```r
################################################################
########################      EN R       #######################
################################################################
library(robustbase)

alumnos = read.csv("alumnos_scidata.csv")
edades = alumnos$Edad
edades = edades[!is.na(edades)]

resumen = summary(edades)
Q1 = as.numeric(resumen[2])
mediana = as.numeric(resumen[3])
Q3 = as.numeric(resumen[5])

RI = Q3-Q1
MC = mc(edades)

lim_inf_MC = Q1-1.5*exp(-3.5*MC)*RI
lim_sup_MC = Q3+1.5*exp(4*MC)*RI

atipicos_mc = edades[(edades < lim_inf_MC) | (edades > lim_sup_MC)]
alumnos[alumnos$Edad %in% atipicos_mc,]

adjbox(edades, main = "Boxplot Ajustado con MedCouple", ylab = "Edad", col = "lightblue")
```

```python
################################################################
########################    EN PYTHON   ########################
################################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import medcouple

alumnos = pd.read_csv("alumnos_scidata.csv")
edades = alumnos["Edad"]
edades = edades[~np.isnan(edades)]

resumen = alumnos["Edad"].describe()

Q1 = resumen[4]
mediana = resumen[5]
Q3 = resumen[6]

RI = Q3 - Q1
MC = medcouple(edades.to_numpy())
lim_inf_MC = Q1 - 1.5 * np.exp(-3.5 * MC) * RI
lim_sup_MC = Q3 + 1.5 * np.exp(4 * MC) * RI

atipicos_mc = edades[(edades < lim_inf_MC) | (edades > lim_sup_MC)]
alumnos[alumnos["Edad"].isin(atipicos_mc)]

############### Creación del boxplot ajustado

edades_filtradas = edades[(edades >= lim_inf_MC) & (edades <= lim_sup_MC)] 

# Crear boxplot con los bigotes ajustados
fig, ax = plt.subplots()

boxprops = dict(facecolor="lightblue", linewidth=1.5)
medianprops = dict(color="red", linewidth=1.5)
whiskerprops = dict(color="blue", linestyle="dashed", linewidth=1.5)

# Crear el boxplot ajustado sin atípicos
ax.boxplot(edades_filtradas, vert=True, patch_artist=True, boxprops=boxprops,
           whiskerprops=whiskerprops, capprops=dict(color="black", linewidth=1.5),
           medianprops=medianprops, flierprops=dict(marker='o', color='red', alpha=0))

# Dibujar los valores atípicos ajustados manualmente
ax.scatter(np.ones_like(atipicos_mc), atipicos_mc, color="red", alpha=0.7, label="Atípicos ajustados")

# Etiquetas y título
ax.set_xticks([])
ax.set_title("Boxplot Ajustado con MedCouple")
ax.set_ylabel("Edad")
ax.legend()

plt.show()


```